In [1]:
from icecube.models import IceCubeModelEncoderMAT, VonMisesFisher3DLoss, IceCubeModelEncoderV2
from icecube.dataset import HuggingFaceDatasetV6
from icecube.utils import collate_fn_graphv0, collate_fn, get_score_vector
from datasets import  load_from_disk
from torch.utils.data import DataLoader
import torch
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch import nn

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
md = IceCubeModelEncoderV2().eval()
ds = HuggingFaceDatasetV6(load_from_disk('/opt/slh/icecube/data/hf_cashe/batch_1.parquet'))
dl = DataLoader(ds, batch_size=64, shuffle=True, num_workers=4, collate_fn=collate_fn)
with torch.no_grad():
    for x in dl:
        out = md(x)
        break

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [3]:
VonMisesFisher3DLoss()(out, x['label'])

tensor(2.5550)

In [4]:
get_score_vector(out, x['label'])

array(1.4765253, dtype=float32)

In [ ]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.abs(np.arccos(scalar_prod))

In [ ]:
exp_name = 'EXP_07'

!ls {Path('RESULTS/')/exp_name}

In [ ]:
#function that sort the files by the number in the name and read them
def read_oof_files(exp_name, epoch = 9):
    oof_files = sorted((Path('RESULTS/')/exp_name).glob("*OOF*.csv"), 
                 key = lambda x: int(x.stem.split('_')[-1].split('.')[0]))
    
    oof = pd.read_csv(oof_files[epoch])

    oof['metric'] = angular_dist_score(oof['azimuth_gt'], oof['zenith_gt'], 
                  oof['azimuth_pred'], oof['zenith_pred'])
    return oof

#function that read csv files not OOF and sorted by the number in the name
def read_logs(exp_name):
    files = list((Path('RESULTS/')/exp_name).glob("*.csv"))
    files = [file for file in files if 'OOF' not in file.name]
    files = sorted(files, key = lambda x: int(x.stem.split('_')[-1].split('.')[0]))
    logs = [pd.read_csv(file) for file in files]
    logs = pd.concat(logs)
    return logs

In [ ]:
#oof = read_oof_files(exp_name, epoch = 9)

In [ ]:
import config

In [ ]:
read_logs('EXP_07')

In [ ]:
read_logs('EXP_08')

In [ ]:
get_config_as_dict(config.BASELINE_EMBED_V3)

In [ ]:
torch.__version__

In [2]:
!pip uninstall torch-geometric -y

Found existing installation: torch-geometric 2.2.0
Uninstalling torch-geometric-2.2.0:
  Successfully uninstalled torch-geometric-2.2.0
